In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import cvxpy as cp

import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.optim as optim
import os
import random

import sys
sys.path.insert(0, './mlopt-micp')
sys.path.insert(0, './mlopt-micp/cartpole')

import optimizer
from problem import Cartpole
from src.ae import Encoder, get_cartpole_encoder

In [3]:
def euclidean_dist(x,y):
    # x: NxD
    # y: MxD
    n = x.size(0)
    m = y.size(0)
    d = x.size(1)
    assert d == y.size(1)
    
    x = x.unsqueeze(1).expand(n, m, d)
    y = y.unsqueeze(0).expand(n, m, d)
    return torch.pow(x-y, 2).sum(2)

In [4]:
pp = Cartpole()

In [5]:
print('Total number of classes: {}'.format(pp.n_strategies))
print('Length of feature vector: {}'.format(pp.n_features))

Total number of classes: 581
Length of feature vector: 13


In [22]:
dim_in, dim_z = pp.n_features, 4#pp.n_strategies

enc = get_cartpole_encoder(dim_in, dim_z).cuda()
enc(torch.from_numpy(pp.features[:2]).float().cuda())

# training parameters
TRAINING_ITERATIONS = int(5000)
BATCH_SIZE = int(10)
CHECKPOINT_AFTER = int(1250)
SAVEPOINT_AFTER = int(2500)

rand_idx = list(np.arange(0, pp.n_strategies-1))

indices = [rand_idx[ii * BATCH_SIZE:(ii + 1) * BATCH_SIZE] for ii in range((len(rand_idx) + BATCH_SIZE - 1) // BATCH_SIZE)]
random.shuffle(indices)

enc_dict = {}
str_dict = {}
for ii in range(len(pp.features)):
    str_idx = int(pp.labels[ii,0])
    str_dict[ii] = str_idx
    if str_idx in enc_dict.keys():
        enc_dict[str_idx] += [ii]
    else:
        enc_dict[str_idx] = [ii]
        
feats = torch.from_numpy(pp.features).float().cuda()

pp.training_batch_percentage = 1.
pp.construct_strategies()
strat_lookup = {}
for k, v in pp.strategy_dict.items():
    strat_lookup[v[0]] = v[1:]
pp.training_batch_percentage = 0.9
pp.n_evals = 5

In [23]:
#nearest neighbors
train_set_length = int(pp.training_batch_percentage*pp.n_probs)
Y = feats[:train_set_length,:]

#classifier
def nn_classifier(x,Y):
    dist_inds = torch.argsort(torch.cdist(Y,x[None,:]),dim=0).cpu().numpy()
    strats_sorted = pp.labels[dist_inds,0].astype(int)
    _, unique_inds = np.unique(strats_sorted,return_index=True)
    return np.concatenate([strats_sorted[index] for index in sorted(unique_inds)])

nn_classifier(feats[train_set_length+1,:],Y);

In [24]:
#test script
n_train_strategies = pp.n_strategies #store how many strats in train set
c_k = torch.zeros((n_train_strategies,4)) 
embeddings = enc(feats) #embed training points
for ii in range(n_train_strategies): #compute train centroids
    inds = enc_dict[ii]
    c_k[ii,:] = torch.mean(embeddings[inds,:],axis=0).cuda()

#compute strategy dictionary for all problems
pp.training_batch_percentage = 1.
pp.construct_strategies()
strat_lookup = {}
for k, v in pp.strategy_dict.items():
    strat_lookup[v[0]] = v[1:]

#setup for test
test_feats = torch.from_numpy(pp.features[int(0.9*pp.n_probs):,:]).float().cuda()
test_enc = enc(test_feats).cuda()
test_dists = torch.cdist(test_enc,c_k.cuda()).detach().cpu().numpy()
test_start = int(0.9*pp.n_probs)
n_test = int(0.1*pp.n_probs)
ind_max = np.argsort(test_dists)[:,:pp.n_evals]
feasible = np.zeros(n_test)
costs = np.zeros(n_test)

In [25]:
prob_success = False

for ii in range(n_test):
    strats_sorted = nn_classifier(feats[test_start+ii,:],Y);
    for jj in range(pp.n_evals):
        y_guess = strat_lookup[strats_sorted[jj]]
    #y_guess = strat_lookup[int(pp.labels[ii,0])]
        #try:
        prob_success, cost, solve_time = pp.solve_mlopt_prob_with_idx(ii, y_guess)
        if prob_success:
            feasible[ii] = 1.
            costs[ii] = cost
            print('Succeded at {} with {} tries'.format(ii,jj+1))
            break
        #except:
            #print('mosek failed at '.format(ii))

Succeded at 3 with 1 tries
Succeded at 6 with 1 tries
Succeded at 13 with 1 tries
Succeded at 19 with 1 tries
Succeded at 30 with 3 tries
Succeded at 32 with 1 tries
Succeded at 34 with 4 tries
Succeded at 45 with 2 tries
Succeded at 46 with 2 tries
Succeded at 51 with 1 tries
Succeded at 52 with 5 tries
Succeded at 60 with 1 tries
Succeded at 62 with 5 tries
Succeded at 69 with 5 tries
Succeded at 70 with 2 tries
Succeded at 76 with 4 tries
Succeded at 93 with 1 tries
Succeded at 94 with 5 tries
Succeded at 95 with 1 tries
Succeded at 96 with 1 tries
Succeded at 98 with 2 tries
Succeded at 103 with 1 tries
Succeded at 106 with 1 tries
Succeded at 107 with 2 tries
Succeded at 116 with 1 tries
Succeded at 124 with 1 tries
Succeded at 127 with 1 tries
Succeded at 138 with 5 tries
Succeded at 143 with 2 tries
Succeded at 149 with 4 tries
Succeded at 154 with 1 tries
Succeded at 156 with 2 tries
Succeded at 157 with 1 tries
Succeded at 158 with 2 tries
Succeded at 159 with 1 tries
Succeded

SolverError: Solver 'MOSEK' failed. Try another solver, or solve with verbose=True for more information.

In [26]:
global_acc = sum(sum(np.equal(ind_max,pp.labels[test_start:,0][:,None])))/(0.1*pp.n_probs)
global_acc

0.1315

In [28]:
sum(feasible)/ii

0.2754151478331308